In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.tree import DecisionTreeClassifier

In [2]:
churn_data = pd.read_csv('Customer-Churn.csv')
churn_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
churn_data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [6]:
churn_data['TotalCharges'] = pd.to_numeric(churn_data['TotalCharges'], errors='coerce')

In [7]:
data = churn_data.dropna(axis='index')

In [8]:
data.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [9]:
num_data = data.select_dtypes(include=['int64', 'float64'])
num_data.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65


In [17]:
cat_data = data.select_dtypes(include=['object'])
cat_data = cat_data.drop(['Churn'], axis=1)
cat_data

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month
...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,One year
7039,Female,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,One year
7040,Female,Yes,Yes,No,Yes,No,No,No,No,No,Month-to-month
7041,Male,Yes,No,Yes,No,No,No,No,No,No,Month-to-month


In [11]:
dummy_data = pd.get_dummies(cat_data,drop_first=True)
dummy_data.head()

,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year
0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
data['Churn'].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

In [15]:
data_sm = pd.concat([num_data,data['Churn'],dummy_data],axis=1)
smote = SMOTE()
y = data_sm['Churn']
X = data_sm.drop(['Churn'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)

unique, counts = np.unique(y_sm, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 5163]
 ['Yes' 5163]]


In [16]:
transformer = StandardScaler().fit(X_sm)
X_sm_standarized = transformer.transform(X_sm)
X_sm_standarized = pd.DataFrame(X_sm_standarized)
X_sm_standarized.columns = X_sm_standarized.columns.astype(str)
X_sm_standarized.rename(columns={'0':'SeniorCitizen', '1':'tenure', '2':'MonthlyCharges', '3':'TotalCharges'}, inplace=True)
X_sm_standarized.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.396364,-1.117334,-1.344643,-0.929541,-0.876788,1.279141,-0.524978,-2.973829,-0.436253,-0.511144,...,-0.436253,-0.615336,-0.436253,-0.514307,-0.436253,-0.750732,-0.436253,-0.75152,-0.418983,-0.444717
1,-0.396364,0.261466,-0.397718,-0.076771,1.140527,-0.781775,-0.524978,0.336267,-0.436253,1.956397,...,-0.436253,1.625129,-0.436253,-0.514307,-0.436253,-0.750732,-0.436253,-0.75152,2.386734,-0.444717
2,-0.396364,-1.075552,-0.506038,-0.893635,1.140527,-0.781775,-0.524978,0.336267,-0.436253,1.956397,...,-0.436253,-0.615336,-0.436253,-0.514307,-0.436253,-0.750732,-0.436253,-0.75152,-0.418983,-0.444717
3,-0.396364,0.721066,-0.909617,-0.099126,1.140527,-0.781775,-0.524978,-2.973829,-0.436253,1.956397,...,-0.436253,1.625129,-0.436253,1.944365,-0.436253,-0.750732,-0.436253,-0.75152,2.386734,-0.444717
4,-0.396364,-1.075552,0.082733,-0.873687,-0.876788,-0.781775,-0.524978,0.336267,-0.436253,-0.511144,...,-0.436253,-0.615336,-0.436253,-0.514307,-0.436253,-0.750732,-0.436253,-0.75152,-0.418983,-0.444717


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_sm_standarized, y_sm, test_size=0.3, random_state=100)

In [19]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

In [20]:
predictions_lm = classification.predict(X_test)

In [21]:
print("The accuracy of the model on test set is: %4.2f " % accuracy_score(y_test, predictions_lm))

The accuracy of the model on test set is: 0.83 


In [22]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions_dt = model.predict(X_test)
print("The accuracy of the model is: {:4.2f}".format(model.score(X_test, y_test)))

The accuracy of the model is: 0.78


In [24]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_resample(X, y)

unique, counts = np.unique(y_tl, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 4596]
 ['Yes' 1869]]


In [25]:
transformer = StandardScaler().fit(X_tl)
X_tl_standarized = transformer.transform(X_tl)
X_tl_standarized = pd.DataFrame(X_tl_standarized)
X_tl_standarized.columns = X_tl_standarized.columns.astype(str)
X_tl_standarized.rename(columns={'0':'SeniorCitizen', '1':'tenure', '2':'MonthlyCharges', '3':'TotalCharges'}, inplace=True)
X_tl_standarized.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.43734,0.035998,-0.233070,-0.181635,0.989690,-0.972987,-0.658876,0.328008,-0.545593,1.581824,...,-0.545593,1.386136,-0.545593,-0.636258,-0.545593,-0.781417,-0.545593,-0.791464,1.934739,-0.57872
1,-0.43734,-1.259172,-0.335207,-0.962250,0.989690,-0.972987,-0.658876,0.328008,-0.545593,1.581824,...,-0.545593,-0.721430,-0.545593,-0.636258,-0.545593,-0.781417,-0.545593,-0.791464,-0.516866,-0.57872
2,-0.43734,0.481213,-0.715750,-0.202998,0.989690,-0.972987,-0.658876,-3.048702,-0.545593,1.581824,...,-0.545593,1.386136,-0.545593,1.571690,-0.545593,-0.781417,-0.545593,-0.791464,1.934739,-0.57872
3,-0.43734,-1.259172,0.219958,-0.943188,-1.010418,-0.972987,-0.658876,0.328008,-0.545593,-0.632182,...,-0.545593,-0.721430,-0.545593,-0.636258,-0.545593,-0.781417,-0.545593,-0.791464,-0.516866,-0.57872
4,-0.43734,-1.016328,1.173787,-0.650087,-1.010418,-0.972987,-0.658876,0.328008,-0.545593,-0.632182,...,-0.545593,1.386136,-0.545593,-0.636258,-0.545593,1.279726,-0.545593,1.263482,-0.516866,-0.57872


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_tl_standarized, y_tl, test_size=0.3, random_state=100)


In [27]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

In [28]:
predictions_lm = classification.predict(X_test)


In [29]:
print("The accuracy of the model on test set is: %4.2f " % accuracy_score(y_test, predictions_lm))


The accuracy of the model on test set is: 0.81 


In [30]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions_dt = model.predict(X_test)
print("The accuracy of the model is: {:4.2f}".format(model.score(X_test, y_test)))

The accuracy of the model is: 0.73


In [32]:
X_tl2, y_tl2 = tl.fit_resample(X_tl, y_tl)

unique, counts = np.unique(y_tl2, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 4429]
 ['Yes' 1869]]
